In [2]:
import nltk  
import numpy as np  
import random  
import string

import bs4 as bs  
import urllib.request  
import re  
import pandas as pd

In [9]:
df = pd.read_excel('Comments.xlsx')

article_text = ''

for para in df['Comment']:  
    article_text += str(para)

In [ ]:
print(article_text)

In [11]:
corpus = nltk.sent_tokenize(article_text)

In [12]:
for i in range(len(corpus )):
    corpus [i] = corpus [i].lower()
    corpus [i] = re.sub(r'\W',' ',corpus [i])
    corpus [i] = re.sub(r'\s+',' ',corpus [i])

In [13]:
print(len(corpus))

2821


In [16]:
print(corpus[1500])

she can also set up a small office home network of atleast 20 users sarah has acquired hands on skills to set up a lan with a number of pcs 


In [17]:
wordfreq = {}
for sentence in corpus:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [18]:
print(wordfreq)

{'djfjkdfjkjkffdk': 1, 'editedfaith': 1, 'has': 1070, 'exhibited': 32, 'enthusiasm': 13, 'in': 1162, 'taking': 12, 'on': 1232, 'the': 4175, 'project': 196, 'at': 287, 'hand': 42, 'she': 811, 'already': 14, 'shown': 21, 'promising': 57, 'signs': 1, 'understanding': 83, 'core': 4, 'concepts': 67, 'of': 1624, 'software': 113, 'engineering': 8, 'i': 592, 'look': 24, 'forward': 69, 'discovering': 2, 'more': 266, 'as': 400, 'time': 321, 'goes': 5, 'he': 1458, 'now': 201, 'understood': 18, 'structure': 14, 'grails': 1, 'and': 2946, 'how': 163, 'different': 79, 'components': 13, 'relate': 3, 'understands': 12, 'process': 29, 'getting': 25, 'api': 33, 's': 584, 'from': 233, 'mtnthe': 1, 'intern': 107, 'was': 761, 'oriented': 15, 'ict': 28, 'setup': 28, 'infrastructure': 9, 'soroti': 5, 'university': 14, 'drafted': 1, 'workplan': 2, 'started': 89, 'off': 44, 'internship': 502, 'simon': 10, 'encouraged': 10, 'to': 3531, 'take': 47, 'seriously': 3, 'there': 51, 'a': 1553, 'lot': 44, 'learn': 215, 

In [19]:
sentence_vectors = np.asarray(wordfreq)

In [26]:
df_wordfreq = pd.DataFrame(wordfreq.items(), columns=['Word', 'Freq'])

In [29]:
df_wordfreq = df_wordfreq.sort_values('Freq', ascending=False)

df_wordfreq

,Word,Freq
8,the,4175
63,to,3531
36,and,2946
20,of,1624
67,a,1553
...,...,...
4836,ofcourse,1
4837,suggestions,1
4838,competently,1
4839,today,1
